In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics import r2_score, mean_squared_error
from sklearn.linear_model import LinearRegression

In [7]:
# cost or loss function
def cost(Y, Yhat):
    return np.mean((Yhat - Y) ** 2)

In [8]:
path = '../../data/KNN_Linear_Regression/vidu4_lin_reg.txt'

df = pd.read_csv(path, sep=" ")
df.head()

,ID,TUOI,BMI,HA,GLUCOSE,CHOLESTEROL,BEDAYNTM
0,1,56,21,160,14.0,6.00,1.95
1,2,76,18,150,12.0,4.97,1.33
2,3,63,16,160,4.4,6.39,0.83
3,4,78,20,100,4.0,7.00,2.00
4,5,87,20,110,4.6,4.10,1.30


In [9]:
X = df.drop(['ID', 'BEDAYNTM'], axis=1)
y = df['BEDAYNTM']

X_train = X[:80]
y_train = y[:80]

X_test = X[80:]
y_test = y[80:]

In [14]:
y_train = y_train.to_numpy().reshape(-1, 1)
y_test = y_test.to_numpy().reshape(-1, 1)

In [15]:
d0 = 5
d1 = h = 100 # size of hidden layer
d2 = C = 1
# initialize parameters randomly
W1 = 0.01*np.random.randn(d0, d1)
b1 = np.zeros((d1, 1))
W2 = 0.01*np.random.randn(d1, d2)
b2 = np.zeros((d2, 1))

N = X_train.T.shape[1]
eta = 0.001 # learning rate

for i in range(10000):
    ## Feedforward
    Z1 = np.dot(W1.T, X_train.T) + b1
    A1 = np.where(Z1 > 0, Z1, 0.01 * Z1)  # LeakyReLU
    Z2 = np.dot(W2.T, A1) + b2
    Yhat = Z2

    # print loss after each 1000 iterations
    if i %1000 == 0:
        # compute the loss: average cross-entropy loss
        loss = cost(y_train, Yhat)
        print("iter %d, loss: %f" %(i, loss))

    # backpropagation
    E2 = (Yhat - y_train.T)/N
    dW2 = np.dot(A1, E2.T)
    db2 = np.sum(E2, axis = 1, keepdims = True)
    E1 = np.dot(W2, E2)
    E1 = np.where(Z1 > 0, E1, 0.01 * E1)  # Gradient of LeakyReLU
    dW1 = np.dot(X_train.T, E1.T)
    db1 = np.sum(E1, axis = 1, keepdims = True)
    
    # Gradient clipping (To avoid booming gradient)
    clip_value = 1.0
    dW1 = np.clip(dW1, -clip_value, clip_value)
    dW2 = np.clip(dW2, -clip_value, clip_value)
    db1 = np.clip(db1, -clip_value, clip_value)
    db2 = np.clip(db2, -clip_value, clip_value)
    
    # Gradient Descent update
    W1 += -eta*dW1
    b1 += -eta*db1
    W2 += -eta*dW2
    b2 += -eta*db2

iter 0, loss: 1.153158
iter 1000, loss: 0.151161
iter 2000, loss: 0.151270
iter 3000, loss: 0.151394
iter 4000, loss: 0.151515
iter 5000, loss: 0.151641
iter 6000, loss: 0.151752
iter 7000, loss: 0.151921
iter 8000, loss: 0.152012
iter 9000, loss: 0.152110


In [19]:
Z1 = np.dot(W1.T, X_train.T) + b1
A1 = np.maximum(Z1, 0)
Z2 = np.dot(W2.T, A1) + b2

Z2

array([[1.11497719, 1.23998669, 1.07354795, 1.08679278, 1.20380262,
        1.18877061, 1.01981125, 1.04947974, 1.18006742, 1.23083979,
        0.67018851, 1.14544376, 1.22453133, 1.02368053, 0.97461217,
        0.74332005, 1.05975262, 1.22566149, 1.06605535, 1.06686756,
        1.25728034, 1.16697728, 1.37735595, 0.80889336, 1.36306982,
        1.21070689, 0.99490963, 0.94867508, 1.26021516, 0.96376519,
        1.02069571, 1.13204354, 1.13611234, 0.99688824, 0.85768182,
        1.19945584, 1.03741155, 1.06089705, 1.14798274, 1.09071093,
        1.32982296, 1.02360686, 0.76585105, 0.98519522, 1.02839671,
        1.23730652, 0.9256423 , 0.97945776, 1.01762779, 0.74853045,
        1.01135444, 1.13011086, 1.03044775, 1.09680799, 1.22701591,
        1.14640831, 0.79531051, 1.0421316 , 0.78324817, 1.08130627,
        0.98489927, 1.07130752, 0.97317969, 1.08298122, 0.93630852,
        0.88488432, 1.00248139, 1.07940235, 0.87386834, 1.05629865,
        1.16836865, 1.46101701, 1.12893455, 0.92

In [20]:
print("R square:", r2_score(y_train, Z2[0]))
print("MSE:", mean_squared_error(y_train, Z2[0]))

R square: 0.1973020716166839
MSE: 0.10353487604071536


In [21]:
Z1 = np.dot(W1.T, X_test.T) + b1
A1 = np.maximum(Z1, 0)
Z2 = np.dot(W2.T, A1) + b2

Z2

array([[0.89754097, 0.79275009, 1.20561969, 0.94371207, 0.95009381,
        1.04758867, 1.21050124, 1.06368438, 0.99936876, 0.68683721,
        0.79799435, 0.75365748, 1.17834715, 1.05542705, 0.97537861,
        1.01857953, 1.16276825, 1.18606181, 1.31855509, 1.28324927]])

In [22]:
print("R square:", r2_score(y_test, Z2[0]))
print("MSE:", mean_squared_error(y_test, Z2[0]))

R square: 0.20765114323224465
MSE: 0.22939509648218898


In [23]:
linR = LinearRegression()

linR.fit(X_train, y_train)

y_pred = linR.predict(X_test)

print("R square:", r2_score(y_test, y_pred))
print("MSE:", mean_squared_error(y_test, y_pred))

R square: 0.21453893938640334
MSE: 0.22740099167615907
